In [1]:
# Import all required libraries
import py3Dmol
import os
import subprocess
import pandas as pd
import numpy as np
import yaml
import gc

from pathlib import Path
from rdkit import Chem
from rdkit.Chem import Draw, Descriptors, Lipinski, FilterCatalog, rdMolDescriptors
from rdkit import RDLogger

# Disable RDKit warnings
RDLogger.DisableLog('rdApp.*')

# Set thread limits for compatibility
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

# Import MDAnalysis and prolif after setting thread limits
import MDAnalysis as mda
import prolif

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
/home/teaching/miniconda/envs/SMDD_env/lib/python3.10/site-packages/prolif/datafiles.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/home/teaching/miniconda/envs/SMDD_env/lib/python3.10/site-packages/MDAnalysis/topology/tables.py:52: DeprecationWarning: Deprecated in version 2.8.0
MDAnalysis.topology.tables has been moved to MDAnalysis.guesser.tables. This import point will be removed in MDAnalysis version 3.0.0
  warnings.warn(wmsg, category=DeprecationWarning)


## Designing improved inhibitors for the Zika virus NS2B–NS3 protease (PDB: 7I9O) 🦟❌

In this notebook we will act as an early-stage antiviral design team.
Our biological target is the **Zika virus NS2B–NS3 protease**, a serine protease formed by the NS3 catalytic domain together with its NS2B cofactor. This protease is essential for ZIKV replication because it cleaves the viral polyprotein into the individual structural and non-structural proteins the virus needs to assemble and replicate, which makes it a high-value antiviral target. ([Nature][1])
We will use the experimentally solved crystal structure **7I9O**, which captures the ZIKV NS2B–NS3 protease bound to a small-molecule inhibitor. ([rcsb.org][2])

You are given a **core scaffold** derived from a weak hit in that pocket. The atoms in the core are “locked”: we assume they are important for binding. Certain positions on the scaffold are marked with `*`. Those `*` positions are attachment points where we are allowed to grow new **R-groups** to make the molecule bind better and look more like a viable lead.

We will use **LibINVENT**, a scaffold-decorator model, to explore these R-groups. LibINVENT proposes substituents for the `*` sites and we will train/steer it with reinforcement learning. The key lever you control is the **scoring function** in `zika.toml`: you will define what “good” means (for example: reasonable molecular weight, acceptable physicochemical properties, no obvious liabilities, etc.), and LibINVENT will try to generate molecules that satisfy that profile.

Your workflow in this notebook following the same flow as the previous one with many spots for tweaking:

1. **Define a scoring function**
   Edit the TOML so that high score = “this looks like a plausible NS2B–NS3 protease inhibitor and a drug-like small molecule”.
2. **Generate candidates with LibINVENT**
   Run RL to sample decorated molecules starting from the given scaffold.
3. **Triage / down-select**
   You cannot dock thousands of structures. You should prioritise and filter the generated molecules (chemistry sanity, diversity, properties) and choose at most ~100 molecules that are worth docking into 7I9O.
4. **Nominate synthesis candidates**
   From the docked / prioritised set, choose your final **top 10 compounds**. These 10 are the ones you would hand to medicinal chemistry as proposed “next-step” lead ideas against the Zika NS2B–NS3 protease.

[1]: https://www.nature.com/articles/s41467-025-63602-z?utm_source=chatgpt.com "Combined crystallographic fragment screening and deep ..."
[2]: https://www.rcsb.org/structure/7i9o?utm_source=chatgpt.com "7I9O: Group deposition of ZIKV NS2B-NS3 protease in ..."


In [2]:
# make sure output dir exists
Path("output").mkdir(exist_ok=True)

CONFIG_PATH = Path("config/zika.toml").resolve()
LOG_PATH    = Path("output/zika.log").resolve()

In [3]:
# Continue!!